In [3]:
#!pip install pygal\
!pip install python-terrier


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup

In [4]:
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



In [5]:
!pip install Arabic-Stopwords


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 11.3 MB/s eta 0:00:00


In [6]:

#we need to import the following libraries.
import pandas as pd
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150)
import re
from snowballstemmer import stemmer
import arabicstopwords.arabicstopwords as stp
#make your loops show a smart progress meter
from tqdm import tqdm

In [7]:
dataset_links=["https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-01.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-02.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-03.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-04.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-05.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-06.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-07.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-08.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-09.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-10.txt"]








In [8]:
full_data=pd.DataFrame()
for i in tqdm(range(len(dataset_links))):
    tweets=pd.read_csv(dataset_links[i], sep='\t')
    full_data=pd.concat([full_data,tweets],ignore_index=True)
full_data.reset_index(inplace=True,drop=True)
full_data

100%|██████████| 10/10 [00:03<00:00,  2.97it/s]


,tweetID,tweetText
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر"" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn"
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh
...,...,...
49995,561985373048299520,مواسيا الشعب السعودي..حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله #الخبر #السعودية #saudi #ksa
49996,561987332878766081,@al_shalal @F_D_A82 تم تفجير صماخنا
49997,561988825186971650,@aubyazid123 جزاك الله ألف خير ❌ جزاك الله خير ✔️ - كلمة ألف فيها تحجير لخير الله.
49998,561991173360091136,كيف نفّذت «النصرة» عمليّة تفجير الحافلة اللبنانية في دمشق؟ http://t.co/TEmP1Dso1v


In [9]:
full_data["docno"]=full_data["tweetID"].astype(str)
full_data[["docno"]]

,docno
0,549679192804061184
1,549699343666532352
2,549711593487888387
3,549719610459967488
4,549720880717508608
...,...
49995,561985373048299520
49996,561987332878766081
49997,561988825186971650
49998,561991173360091136


we need to clean the tweets before the other processsing steps them (remove the urls, emojies....)

In [10]:
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   accents = re.compile(r'[\u064b-\u0652\u0640]') # harakaat and tatweel (kashida) to remove

   arabic_punc= re.compile(r'[\u0621-\u063A\u0641-\u064A\d+]+') # Keep only Arabic letters/do not remove number
   text=' '.join(arabic_punc.findall(accents.sub('',text)))
   text = text.strip()
   return text

In [11]:
full_data["text"]=full_data["tweetText"].apply(clean)
full_data[['docno','tweetText','text']]

,docno,tweetText,text
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر"" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn",الاعدام لعامل مطعم قتل زميله طعنا في البيادر أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR,الأخبار تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري,عشآن أعطيتك وحده صميم صرت ترمي أعذار حقق العالميةة و أرجع كلمني ي الأياب الانتحاري
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu,النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص شبكة الصين
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh,البحرين ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا
...,...,...,...
49995,561985373048299520,مواسيا الشعب السعودي..حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله #الخبر #السعودية #saudi #ksa,مواسيا الشعب السعودي حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله الخبر السعودية
49996,561987332878766081,@al_shalal @F_D_A82 تم تفجير صماخنا,تم تفجير صماخنا
49997,561988825186971650,@aubyazid123 جزاك الله ألف خير ❌ جزاك الله خير ✔️ - كلمة ألف فيها تحجير لخير الله.,جزاك الله ألف خير جزاك الله خير كلمة ألف فيها تحجير لخير الله
49998,561991173360091136,كيف نفّذت «النصرة» عمليّة تفجير الحافلة اللبنانية في دمشق؟ http://t.co/TEmP1Dso1v,كيف نفذت النصرة عملية تفجير الحافلة اللبنانية في دمشق


In [12]:
#removing Stop Words function
def remove_stopWords(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() :
        if term not in stopWords :
           terms.append(term)
    return " ".join(terms)

In [14]:
full_data["text"]=full_data["text"].apply(remove_stopWords)
full_data[['docno','tweetText','text']]

,docno,tweetText,text
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر"" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn",الاعدام لعامل مطعم قتل زميله طعنا البيادر أيدت محكمة التمييز الحكم الصادر محكمة الجنايات الكبرى والقاضي
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR,الأخبار تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع أجلت محكمة الجنايات بالعاصمة تاريخ لاحق محاكمة سبعة إ
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري,عشآن أعطيتك وحده صميم صرت ترمي أعذار حقق العالميةة أرجع كلمني الأياب الانتحاري
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu,النهدي ثمانية قتلى تفجير انتحاري بسيارة مفخخة معملين للغاز ريف حمص شبكة الصين
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh,البحرين ضبط مطلوبين متورطين التفجير بالعكر الشرقي بقية الموضوع اضغط
...,...,...,...
49995,561985373048299520,مواسيا الشعب السعودي..حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله #الخبر #السعودية #saudi #ksa,مواسيا الشعب السعودي حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة الملك عبدالله الخبر السعودية
49996,561987332878766081,@al_shalal @F_D_A82 تم تفجير صماخنا,تم تفجير صماخنا
49997,561988825186971650,@aubyazid123 جزاك الله ألف خير ❌ جزاك الله خير ✔️ - كلمة ألف فيها تحجير لخير الله.,جزاك الله ألف خير جزاك الله خير كلمة ألف تحجير لخير الله
49998,561991173360091136,كيف نفّذت «النصرة» عمليّة تفجير الحافلة اللبنانية في دمشق؟ http://t.co/TEmP1Dso1v,نفذت النصرة عملية تفجير الحافلة اللبنانية دمشق


In [15]:
#a function to normalize the tweets
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

In [16]:
#we will normalize using our normalize function.
full_data["text"]=full_data["text"].apply(normalize)
full_data[['docno','tweetText','text']]

,docno,tweetText,text
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر"" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn",الاعدام لعامل مطعم قتل زميله طعنا البيادر ايدت محكمه التمييز الحكم الصادر محكمه الجنايات الكبري والقاضي
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR,الاخبار تاجيل محاكمه 7 ارهابيين بسبب غياب الدفاع اجلت محكمه الجنايات بالعاصمه تاريخ لاحق محاكمه سبعه ا
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري,عشان اعطيتك وحده صميم صرت ترمي اعذار حقق العالميهه ارجع كلمني الاياب الانتحاري
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu,النهدي ثمانيه قتلي تفجير انتحاري بسياره مفخخه معملين للغاز ريف حمص شبكه الصين
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh,البحرين ضبط مطلوبين متورطين التفجير بالعكر الشرقي بقيه الموضوع اضغط
...,...,...,...
49995,561985373048299520,مواسيا الشعب السعودي..حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله #الخبر #السعودية #saudi #ksa,مواسيا الشعب السعودي حاكم دبي يبدا جلسه مجلس الوزراء بقراءه الفاتحه الملك عبدالله الخبر السعوديه
49996,561987332878766081,@al_shalal @F_D_A82 تم تفجير صماخنا,تم تفجير صماخنا
49997,561988825186971650,@aubyazid123 جزاك الله ألف خير ❌ جزاك الله خير ✔️ - كلمة ألف فيها تحجير لخير الله.,جزاك الله الف خير جزاك الله خير كلمه الف تحجير لخير الله
49998,561991173360091136,كيف نفّذت «النصرة» عمليّة تفجير الحافلة اللبنانية في دمشق؟ http://t.co/TEmP1Dso1v,نفذت النصره عمليه تفجير الحافله اللبنانيه دمشق


In [17]:
ar_stemmer = stemmer("arabic")
#define the stemming function
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])

In [18]:
full_data['text']=full_data['text'].apply(stem)
full_data[['docno','tweetText','text']]

,docno,tweetText,text
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر"" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn",اعدام لعامل مطعم قتل زميل طعنا بيادر ايد محكم تمييز حكم صادر محكم جنا كبر والقاض
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب الدفاع: أجلت محكمة الجنايات بالعاصمة إلى تاريخ لاحق محاكمة سبعة إ... http://t.co/GM4jmpAWbR,اخبار تاجيل محا 7 ارهابي سبب غياب دفاع اجل محكم جنا عاصمه تاريخ لاحق محا سبع ا
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذار ...حقق العالميةة و أرجع كلمني يَ الأياب الانتحاري,عشان اعطي وحد صميم صرت ترم اعذار حقق عالميهه ارجع كلم اياب انتحار
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مفخخة أمام معملين للغاز في ريف حمص - شبكة الصين http://t.co/r5zFEuzAPu,نهد ثما قتل تفجير انتحار سيار مفخخ معمل غاز ريف حمص شبك صين
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر الشرقي بقية الموضوع اضغط هنا http://t.co/t4A5bNrqyh,بحر ضبط مطلوب متورط تفجير عكر شرق بقي موضوع اضغط
...,...,...,...
49995,561985373048299520,مواسيا الشعب السعودي..حاكم دبي يبدأ جلسة مجلس الوزراء بقراءة الفاتحة على الملك عبدالله #الخبر #السعودية #saudi #ksa,مواسي شعب سعود حاكم دب يبد جلس مجلس وزراء قراء فاتحه ملك عبدالل خبر سعوديه
49996,561987332878766081,@al_shalal @F_D_A82 تم تفجير صماخنا,تم تفجير صماخ
49997,561988825186971650,@aubyazid123 جزاك الله ألف خير ❌ جزاك الله خير ✔️ - كلمة ألف فيها تحجير لخير الله.,جزا الله الف خير جزا الله خير كلم الف تحجير لخير الله
49998,561991173360091136,كيف نفّذت «النصرة» عمليّة تفجير الحافلة اللبنانية في دمشق؟ http://t.co/TEmP1Dso1v,نفذ نصره عمل تفجير حافله لبنانيه دمشق


In [19]:
indexer = pt.DFIndexer("./evetarIndex", overwrite=True)
#as the default id an English tokenizer we will update it by setting it to a non-English tokenizer "UTFTokenizer"
indexer.setProperty("tokeniser", "UTFTokeniser")
index_ref = indexer.index(full_data["text"], full_data["docno"])
index_ref.toString()

<ipython-input-19-029de892f70d>:4: DeprecationWarning: Setting of tokeniser property directly is deprecated
  index_ref = indexer.index(full_data["text"], full_data["docno"])


'./evetarIndex/data.properties'

In [20]:
#we will first load the index
index = pt.IndexFactory.of(index_ref)
#we will call getCollectionStatistics() to check the stats
print(index.getCollectionStatistics().toString())

Number of documents: 50000
Number of terms: 25044
Number of postings: 499706
Number of fields: 0
Number of tokens: 537702
Field names: []
Positions:   false



In [21]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

Streaming output truncated to the last 5000 lines.
نطلق -> term13465 Nt=3 TF=3 maxTF=1 @{0 593077 6} 
نطم -> term19639 Nt=2 TF=2 maxTF=1 @{0 593088 2} 
نطو -> term16198 Nt=1 TF=1 maxTF=1 @{0 593095 6} 
نطيط -> term24694 Nt=3 TF=3 maxTF=1 @{0 593099 4} 
نظ -> term18811 Nt=1 TF=1 maxTF=1 @{0 593108 0} 
نظاف -> term11994 Nt=5 TF=5 maxTF=1 @{0 593111 6} 
نظافه -> term22053 Nt=1 TF=1 maxTF=1 @{0 593124 2} 
نظاكم -> term22459 Nt=1 TF=1 maxTF=1 @{0 593128 2} 
نظال -> term21680 Nt=1 TF=1 maxTF=1 @{0 593132 2} 
نظام -> term1493 Nt=127 TF=132 maxTF=2 @{0 593136 2} 
نظر -> term284 Nt=79 TF=81 maxTF=2 @{0 593375 7} 
نظرا -> term16982 Nt=3 TF=3 maxTF=1 @{0 593529 3} 
نظرت -> term19770 Nt=1 TF=1 maxTF=1 @{0 593537 7} 
نظف -> term21783 Nt=1 TF=1 maxTF=1 @{0 593541 7} 
نظل -> term21908 Nt=1 TF=1 maxTF=1 @{0 593545 7} 
نظم -> term1053 Nt=15 TF=15 maxTF=1 @{0 593549 7} 
نظير -> term12356 Nt=51 TF=51 maxTF=1 @{0 593585 3} 
نظيف -> term13931 Nt=21 TF=21 maxTF=1 @{0 593668 5} 
نع -> term19480 Nt=1 TF=1 max

In [40]:
# prompt: Select any document from the collection and check which of its terms appear in the index?
doc_id = 1000
document_text = full_data.loc[doc_id, "text"]

document_text = remove_stopWords(document_text)
document_text = normalize(document_text)
document_text = stem(document_text)


terms = document_text.split()
#check which of its terms appear in the index
for term in terms:
    try:
        #check if the term exists in the index
        if term in index.getLexicon():
            print(f"Term '{term}' found in the index")
        else:
            print(f"Term '{term}' not found in the index")
    except:
        print(f"Error occurred while checking term '{term}'")


Term 'قس' found in the index
Term 'كنيس' found in the index
Term 'قصر' found in the index
Term 'دوبار' not found in the index
Term 'زواج' found in the index
Term 'مشروع' found in the index
Term 'فاشل' found in the index
Term 'قدر' found in the index
Term 'فول' found in the index
Term 'انجح' found in the index
Term 'يوم' found in the index
Term 'سابع' found in the index
